# Install packages

In [ ]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive") 
!pip install neo
!pip install matplotlib_scalebar
!pip install ffmpeg
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2

# Import packages

In [ ]:
# if using google drive
%cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

    
import sys


from data_wrangling import specific_utils, process_monkey_information, base_processing_class
from pattern_discovery import pattern_by_trials, pattern_by_points, make_ff_dataframe, ff_dataframe_utils, pattern_by_trials, pattern_by_points, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, plot_decision_making, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_helper_class, GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class, GUAT_and_TAFT
from decision_making_analysis import free_selection, replacement, trajectory_info
from machine_learning.RL.env_related import env_for_lstm, env_utils, base_env, collect_agent_data_utils
from machine_learning.RL.lstm import GRU_functions, LSTM_functions
from machine_learning.RL.SB3 import interpret_neural_network, sb3_for_multiff_class, rl_for_multiff_utils, SB3_functions
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.matplotlib_tools import plot_trials, plot_polar, additional_plots, plot_behaviors_utils, plot_statistics
from visualization.animation import animation_func, animation_utils, animation_class
from planning_analysis.show_planning import nxt_ff_utils, show_planning_class, show_planning_utils
from planning_analysis.show_planning.cur_vs_nxt_ff import cvn_helper_class, find_cvn_utils, plot_cvn_class, plot_cvn_utils, plot_monkey_heading_helper_class, cvn_from_ref_class
from planning_analysis.show_planning.cur_vs_nxt_ff import cvn_helper_class, find_cvn_utils, plot_cvn_class, plot_cvn_utils, plot_monkey_heading_helper_class, cvn_from_ref_class
from visualization.plotly_tools import plotly_for_monkey, plotly_for_time_series, plotly_preparation
from visualization.dash_tools import dash_prep_class, dash_utils, dash_utils


from importlib import reload
from eye_position_analysis import eye_positions
import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from os.path import exists
import seaborn as sns
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import itertools
import plotly.express as px
from scipy.stats import rankdata
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, hamming_loss, multilabel_confusion_matrix, fbeta_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from math import pi
from scipy.ndimage import gaussian_filter1d
from statsmodels.graphics.tsaplots import plot_acf
import warnings
import os, sys, sys
from sklearn.exceptions import ConvergenceWarning


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 101



# Basic data

## Retrieve monkey data

In [ ]:
PLAYER = "monkey"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
data_item = animation_class.AnimationClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()
data_item.make_PlotTrials_args()

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe

ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
cluster_around_target_indices = data_item.cluster_around_target_indices
ff_caught_T_new = data_item.ff_caught_T_new
caught_ff_num = len(ff_caught_T_new)
ff_flash_sorted = data_item.ff_flash_sorted
ff_flash_end_sorted = data_item.ff_flash_end_sorted
max_point_index = data_item.max_point_index
min_point_index = data_item.min_point_index




data_item.make_or_retrieve_target_clust_last_vis_df()
target_clust_last_vis_df = data_item.target_clust_last_vis_df
target_clust_last_vis_df['ff_index'] = target_clust_last_vis_df['target_index']


PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_new)


plot_polar_args = (monkey_information,
                    ff_dataframe, 
                    ff_life_sorted,
                    ff_real_position_sorted,
                    ff_caught_T_new,
                    ff_flash_sorted,)


trial_total_num = 2
PLAYER = "monkey"

classic_plot_kwargs = {'player': PLAYER,
                       'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True,
                       'show_eye_positions': True,
                       'show_eye_positions_on_the_right': True,
                       'show_connect_path_eye_positions': True,
                       #=== below is different from animation_plot_kwargs
                       'hitting_arena_edge_ok': False,
                       'trial_too_short_ok': False}

combined_plot_kwargs = {'player': PLAYER,
                        'combined_plot': True,
                        'show_alive_fireflies': False,
                        'show_title': False,
                        'show_start': False}

plot_chunks_kwargs = {'player': PLAYER,
                      'show_stops': True,
                      'show_believed_target_positions': True,
                      'show_reward_boundary': True,
                      'show_scale_bar': True,
                      'hitting_arena_edge_ok': True,
                      'trial_too_short_ok': True,
                      'show_connect_path_ff': True,
                      'trail_color_var': 'abs_ddw',
                      'show_colorbar': True}


plot_polar_kwargs = {'rmax': 400,
                    'show_visible_ff': True,
                    'hitting_arena_edge_ok': True,
                    'return_axes': True,
                    'colors_show_overall_time': True,
                    'show_ff_in_memory': True,
                    'show_target_at_being_caught': True
}

print("player is", PLAYER)

animation_plot_kwargs = classic_plot_kwargs.copy()
animation_plot_kwargs['as_part_of_animation'] = True
animation_plot_kwargs['show_eye_positions_on_the_right'] = False   
animation_plot_kwargs['hitting_arena_edge_ok'] = True
animation_plot_kwargs['trial_too_short_ok'] = True
animation_plot_kwargs['images_dir'] = None


  

data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)


plotting_kwargs = {'player': 'monkey',
 'show_stops': True,
 'show_believed_target_positions': True,
 'show_reward_boundary': True,
 'show_scale_bar': True,
 'hitting_arena_edge_ok': True,
 'trial_too_short_ok': True,
 'show_connect_path_ff': True,
 'vary_color_for_connecting_path_ff': True,
 'show_points_when_ff_stop_being_visible': False,
 #'show_connect_path_ff_memory': True,
 'show_alive_fireflies': False,
 'show_visible_fireflies': True,
 'show_in_memory_fireflies': True,
 'connect_path_ff_max_distance': 500,
 #'show_connect_path_ff_except_targets': True,
 'adjust_xy_limits': True,
 'show_null_agent_trajectory': True,
 'show_only_ff_that_monkey_has_passed_by_closely': True,
 'show_null_trajectory_reaching_boundary_ok': False,
 'vary_color_for_connecting_path_ff': False}

In [ ]:
# PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_new)


# plot_polar_args = (monkey_information,
#                     ff_dataframe, 
#                     ff_life_sorted,
#                     ff_real_position_sorted,
#                     ff_caught_T_new,
#                     ff_flash_sorted,)

# data_item.make_PlotTrials_args()
# data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)


## ff_basic_info

In [ ]:
# make ff_basic_info
num_missing_elements = len(ff_real_position_sorted) - len(ff_caught_T_new)
ff_caught_T_new_extended = np.concatenate([ff_caught_T_new, np.repeat(np.nan, num_missing_elements)])
ff_believed_position_sorted_extended = np.concatenate([ff_believed_position_sorted, np.repeat(np.nan, num_missing_elements*2).reshape(-1,2)])

ff_basic_info_bundle = (ff_real_position_sorted, ff_believed_position_sorted_extended, ff_life_sorted, ff_caught_T_new_extended, ff_flash_end_sorted)

ff_basic_info = pd.DataFrame({'ff_index': range(len(ff_real_position_sorted)),
                            'caught_T': ff_caught_T_new_extended,
                            'real_x': ff_real_position_sorted[:,0],
                            'real_y': ff_real_position_sorted[:,1],
                            'believed_x': ff_believed_position_sorted_extended[:,0],
                            'believed_y': ff_believed_position_sorted_extended[:,1],
                            'life_start': ff_life_sorted[:,0],
                            'life_end': ff_life_sorted[:,1],
                            'flash_end': ff_flash_end_sorted,

})


ff_basic_info.head(3)

## plotting kwargs

In [ ]:
null_arcs_plotting_kwargs = {'player': 'monkey',
    'show_stops': True,
    'show_believed_target_positions': True,
    'show_reward_boundary': True,
    'show_scale_bar': True,
    'hitting_arena_edge_ok': True,
    'trial_too_short_ok': True,
    'show_connect_path_ff': False,
    'vary_color_for_connecting_path_ff': True,
    'show_points_when_ff_stop_being_visible': False,
    'show_alive_fireflies': False,
    'show_visible_fireflies': True,
    'show_in_memory_fireflies': True,
    'connect_path_ff_max_distance': 500,
    'adjust_xy_limits': True,
    'show_null_agent_trajectory': False,
    'show_only_ff_that_monkey_has_passed_by_closely': False,
    'show_null_trajectory_reaching_boundary_ok': False,
    'zoom_in': False,
    'truncate_part_before_crossing_arena_edge': True}

In [ ]:
stop!

# CurvatureOfPath class

## best_arc_df

In [ ]:
reload(show_null_trajectory)
reload(curvature_utils)
reload(find_best_arc)



data_item.make_curvature_df([-25, 25])
data_item.find_opt_arc_info_from_curvature_df()
data_item.make_best_arc_df()
data_item.add_column_monkey_passed_by_to_best_arc_df()
data_item.get_elements_for_plotting()

best_arc_df = data_item.best_arc_df.copy()

In [ ]:
curvature_df = data_item.curvature_df.copy()

In [ ]:
curvature_df[['point_index', 'ff_index', 'abs_curv_diff', 'ff_distance', 'ff_angle',
                        'opt_arc_radius', 'opt_arc_measure', 'opt_arc_length']]

## new best_arc_df (only include ff that monkey will pass by)

In [ ]:
reload(curvature_utils)
reload(find_best_arc)

curvature_df = curvature_utils.add_column_monkey_passed_by_to_curvature_df(curvature_df, ff_dataframe, monkey_information)
curvature_utils.clean_curvature_info(curvature_df)
curvature_df_valid = curvature_df[curvature_df['monkey_passed_by'] == True].copy()
best_arc_df, best_arc_original_columns = find_best_arc.make_best_arc_df(curvature_df_valid, monkey_information, ff_real_position_sorted)

## null_arc_info_for_plotting

In [ ]:
reload(show_null_trajectory)
null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(best_arc_df, monkey_information, column_for_color=None)
#null_arc_info_for_plotting = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(best_arc_df, monkey_information, column_for_color='diff_percentile_in_decimal')

# also get null_arc_to_center_info_for_plotting
all_point_index = best_arc_df.point_index.values
all_ff_index = best_arc_df.ff_index.values
remaining_index_of_array, remaining_all_ff_index, remaining_all_point_index = show_null_trajectory.eliminate_invalid_ff_for_null_arc(all_ff_index, all_point_index, ff_real_position_sorted, monkey_information)
null_arc_to_center_info_for_plotting = show_null_trajectory.find_and_package_arc_to_center_info_for_plotting(remaining_all_point_index, remaining_all_ff_index, monkey_information, ff_real_position_sorted)

## pretty_null_arc_plot_kwargs

In [ ]:
plt.rcParams['figure.figsize'] = [9, 9]
best_arc_df_sub = best_arc_df[best_arc_df['monkey_passed_by'] == True].copy()
best_arc_df_sub.sort_values(by=['time'], inplace=True)
all_intended_target_id = best_arc_df_sub.intended_target_id.unique()  
all_intended_target_id_index_pairs = best_arc_df_sub[['intended_target_id', 'ff_index']].drop_duplicates()
PlotTrials_args = data_item.PlotTrials_args

pretty_null_arc_plot_kwargs = {'player': 'monkey',
        'show_believed_target_positions': True,                       
        'show_stops': True,
        'show_reward_boundary': True,
        'show_scale_bar': True,
        'hitting_arena_edge_ok': True,
        'trial_too_short_ok': True,
        'show_alive_fireflies': False,
        'adjust_xy_limits': True,
        'zoom_in': True,
        'minimal_margin': 60,
        }

In [ ]:
stop!

# Visualizations

## plot all arcs

In [ ]:
current_i = 0

In [ ]:
# we want to plot the arcs here...with opt_arc_radius and opt_arc_measure
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
half_length_of_time_window = 0.1
epsilon = 2
max_num_plot_to_make = 10
duration_for_each_chunk = 2
time_samples = range(int(ff_caught_T_new[0]), int(ff_caught_T_new[-1]), duration_for_each_chunk)


plt.rcParams['figure.figsize'] = [10, 10]   
plotting_kwargs_temp = plotting_kwargs.copy()
plotting_kwargs_temp['show_only_ff_that_monkey_has_passed_by_closely'] = False
plotting_kwargs_temp['show_null_agent_trajectory'] = False
plotting_kwargs_temp['vary_color_for_connecting_path_ff'] = True
plotting_kwargs_temp['show_points_when_ff_stop_being_visible'] = False
plotting_kwargs_temp['show_alive_fireflies'] = False


#=============  below are temporary:
plotting_kwargs_temp['show_connect_path_ff'] = False
plotting_kwargs_temp['zoom_in'] = True
plotting_kwargs_temp['adjust_xy_limits'] = True
plotting_kwargs_temp['show_alive_fireflies'] = False
plotting_kwargs_temp['truncate_part_before_crossing_arena_edge'] = True
plotting_kwargs_temp['rotation_matrix'] = None
plot_counter = 0


# need to initiate a value for current_i for using this cell for the first time
num_trials = 1


for i in range(len(time_samples))[current_i:]:

    time = time_samples[i]
    current_i = i

    duration_to_plot = [time, time+duration_for_each_chunk]
    duration_to_plot = [1850, 1852]
    relevant_point_index = monkey_information.loc[monkey_information['time'].between(duration_to_plot[0], duration_to_plot[1])].index.values.astype(int)
    returned_info = plot_trials.PlotTrials(
                duration_to_plot, 
                *data_item.PlotTrials_args,
                **plotting_kwargs_temp,        
                )
    axes = fig.axes[0]

    R = returned_info['rotation_matrix']
    axes = returned_info['axes']
    whether_plotted = returned_info['whether_plotted']

    if whether_plotted == True:
        # To plot null arcs
        x0, y0 = 0, 0
        axes, whether_plotted = show_null_trajectory.plot_null_arcs_from_best_arc_df(axes, relevant_point_index, null_arc_info_for_plotting, x0=0, y0=0, rotation_matrix=R, polar=False)

        plt.show()
        
        plot_counter += 1
        if plot_counter == max_num_plot_to_make:
            break

    break

## connect_path_ff_based_on_intended_targets

In [ ]:
current_i = 0
show_percentile_by_color = True


In [ ]:
color = matplotlib.colormaps['viridis'](0.99)
plt.figure(figsize=(4,2))
plt.plot([0, 1], [0, 1], color=color)
plt.show()


In [ ]:
duration_for_each_chunk = 2
time_samples = range(int(ff_caught_T_new[0]), int(ff_caught_T_new[-1]), duration_for_each_chunk)
max_num_plot_to_make = 3

plt.rcParams['figure.figsize'] = [7, 7]   
all_intended_target_id = best_arc_df.intended_target_id.unique()  
null_arcs_plotting_kwargs_temp = null_arcs_plotting_kwargs.copy()


plot_counter = 0
for i in range(len(time_samples))[current_i:]:  
    time = time_samples[i]
    fig, axes = show_null_trajectory.make_plots_to_show_monkey_reaction_time(curvature_df,                    
                                                                            null_arcs_plotting_kwargs,
                                                                            data_item.PlotTrials_args,
                                                                            time, 
                                                                            show_percentile_by_color=False,
                                                                            additional_plotting_kwargs={})


    plt.show()
    
    plot_counter += 1
    current_i = i
    if plot_counter == max_num_plot_to_make:
        break

## plot best_arc_ff

In [ ]:
current_i = 100

In [ ]:
current_i = current_i-7

In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(find_best_arc)
reload(show_null_trajectory)

num_intended_targets_to_plot = 3
max_num_plot_to_make = 2 # for each intended_target
plot_counter = 0
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
# set global figure size
plt.rcParams["figure.figsize"] = (9, 9)

# need to initiate a value for current_i for using this cell for the first time
for i in range(len(all_intended_target_id_index_pairs))[current_i:current_i+num_intended_targets_to_plot]:
    current_i = i
    intended_target_id = all_intended_target_id_index_pairs.iloc[i, 0]
    intended_target_ff_index = all_intended_target_id_index_pairs.iloc[i, 1]


    best_arc_df_for_one_id = best_arc_df[best_arc_df.intended_target_id == intended_target_id]
    intended_target_ff_index = best_arc_df_for_one_id.ff_index.iloc[0]

    # if ff_index is valid
    if intended_target_ff_index >= 0: 
        relevant_point_indices = best_arc_df_for_one_id.point_index.values
        duration_to_plot = [best_arc_df_for_one_id.time.min()-0.5, best_arc_df_for_one_id.time.max()+3.6]
        duration_to_plot = show_null_trajectory.eliminate_irrelevant_points_before_or_after_crossing_boundary(duration_to_plot, relevant_point_indices, monkey_information)
        

    show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, best_arc_df, monkey_information, ff_dataframe, null_arc_info_for_plotting, null_arc_to_center_info_for_plotting, PlotTrials_args, pretty_null_arc_plot_kwargs_temp, intended_target_ff_index=intended_target_ff_index,
                                        plot_null_arcs=False, ff_max_distance_to_intended_target=450, ff_max_distance_to_path=400, time_between_every_two_null_arcs=0.2, time_to_begin_plotting_null_arc=None, time_to_end_plotting_null_arc=None, max_num_plot_to_make=max_num_plot_to_make,
                                        show_intended_target=False, show_ff_indices=False, point_indices_to_plot_null_arc=None, ff_indices_to_plot_null_arc=None, whether_mark_path_where_intended_target_has_best_arc_among_all_ff=False, 
                                        )

## Planning
(the plots below are used to analyze planning)

In [ ]:
interocular_dist = 4 if monkey_name == 'monkey_Bruno' else 3 
def compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, interocular_dist=4, additional_function_kwargs, additional_plotting_kwargs):
    print('First Average Angles')
    title = 'First Average Angles'
    monkey_information = eye_positions.average_and_then_convert_eye_positions_in_monkey_information(monkey_information, add_suffix_to_new_columns=False)
    show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs,
                                            additional_plotting_kwargs=additional_plotting_kwargs, title=title)  

    print('Average World Coordinates')
    title = 'Average World Coordinates'
    monkey_information = eye_positions.convert_eye_positions_in_monkey_information(monkey_information, add_left_and_right_eyes_info=True, interocular_dist=interocular_dist)    
    show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs,
                                            additional_plotting_kwargs=additional_plotting_kwargs, title=title) 

    print('Just Left Eye')   
    title = 'Just Left Eye' 
    for column in ['gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_world_x', 'gaze_world_y']:
        monkey_information[column] = monkey_information[column + '_l']
    show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs,
                                            additional_plotting_kwargs=additional_plotting_kwargs, title=title)

    print('Just Right Eye')
    title = 'Just Right Eye'
    for column in ['gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_world_x', 'gaze_world_y']:
        monkey_information[column] = monkey_information[column + '_r']
    show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs,
                                            additional_plotting_kwargs=additional_plotting_kwargs, title=title)
    return

### 138

In [ ]:
additional_plotting_kwargs = {'show_eye_positions': True,
                      'show_eye_positions_on_the_right': False,
                      'show_connect_path_eye_positions': True,

}

pretty_plot_args = (best_arc_df, monkey_information, ff_dataframe, null_arc_info_for_plotting, null_arc_to_center_info_for_plotting, PlotTrials_args)



In [ ]:
reload(plot_behaviors_utils)
reload(plot_trials)
reload(show_null_trajectory)
duration_to_plot=[165.455423, 171.578282]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([62, 413, 60, 61])
intended_target_ff_index = 61
point_indices_to_plot_null_arc = [10147]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()


additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                'plot_null_arcs': True,
                                'ff_max_distance_to_intended_target': 450,   
                                'ff_max_distance_to_path': 400,
                                'time_between_every_two_null_arcs': 0.2,
                                'time_to_begin_plotting_null_arc': None,
                                'time_to_end_plotting_null_arc': None,
                                'max_num_plot_to_make': max_num_plot_to_make,
                                'show_intended_target': False,
                                'show_ff_indices': False,
                                'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}


show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
                               

### 140

In [ ]:
reload(show_null_trajectory)
duration_to_plot=[165.594853, 171.827675]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([149, 62, 413, 60, 61])
intended_target_ff_index = 60
point_indices_to_plot_null_arc = [10140, 10195]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_after_coloring_segments_ff_indices']=np.array([149, 62])

additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
                               


### 851

In [ ]:
reload(show_null_trajectory)
duration_to_plot=[955.054736, 960.2909]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([366, 480, 688, 365])
intended_target_ff_index = 366
point_indices_to_plot_null_arc = [58183]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()


additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
                                


### 851 (2)

In [ ]:
reload(show_null_trajectory)
duration_to_plot=[956.41388, 960.090867]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([366, 480, 688, 365])
intended_target_ff_index = 366
point_indices_to_plot_null_arc = [58270]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()

additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
                                
                                      

### 1029

In [ ]:
# new ID: 1002
reload(show_null_trajectory)
duration_to_plot=[1138.93005, 1144.195605]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([437, 438, 498])
intended_target_ff_index = 438
point_indices_to_plot_null_arc = [69382]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()

additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          
                                      

### 1029 (2)

In [ ]:
# new ID: 1002
reload(show_null_trajectory)
duration_to_plot=[1141.43005, 1145.695605]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([437, 438, 498])
intended_target_ff_index = 438
point_indices_to_plot_null_arc = [69557]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()



additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          



### 1038

In [ ]:
# new ID: 1012
reload(show_null_trajectory)
duration_to_plot=[1152.904394, 1156.843345]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([474, 475, 529])
intended_target_ff_index = 473
point_indices_to_plot_null_arc = [70182]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_believed_target_positions'] = False

additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          
                                       

### 1153

In [ ]:
# new ID: 1121
reload(show_null_trajectory)
duration_to_plot = [1252.452492, 1256.694845]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([488, 535, 487])
intended_target_ff_index = 535
point_indices_to_plot_null_arc = [76299]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()


additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          

### 1190

In [ ]:
# new ID: 1155
reload(show_null_trajectory)
duration_to_plot = [1282.594949, 1287.5386760000001]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([498, 540, 541])
intended_target_ff_index = 498
point_indices_to_plot_null_arc = [78171]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()


additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          

### 1203

In [ ]:
# new ID: 1168
reload(show_null_trajectory)
duration_to_plot = [1295.278694, 1300.635839]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([503, 667])
intended_target_ff_index = 503
point_indices_to_plot_null_arc = [78899]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()


additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

# compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          


### 1203 (2)

In [ ]:
# new ID: 1168
reload(show_null_trajectory)
duration_to_plot = [1295.278694, 1301.635839]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([505, 797, 798, 503, 667])
intended_target_ff_index = 503
point_indices_to_plot_null_arc = [78959, 78999]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()
pretty_null_arc_plot_kwargs_temp['show_connect_path_ff_after_coloring_segments_ff_indices']=np.array([797, 798])



additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          

### 1212

In [ ]:
# new ID: 1176

reload(show_null_trajectory)
duration_to_plot = [1308.543045, 1314.339848]
max_num_plot_to_make = 10
ff_indices_to_plot_null_arc = np.array([667, 505, 686])
intended_target_ff_index = 667
point_indices_to_plot_null_arc = [79680]
pretty_null_arc_plot_kwargs_temp = pretty_null_arc_plot_kwargs.copy()


additional_function_kwargs = {'intended_target_ff_index': intended_target_ff_index,
                                        'plot_null_arcs': True,
                                        'ff_max_distance_to_intended_target': 450,   
                                        'ff_max_distance_to_path': 400,
                                        'time_between_every_two_null_arcs': 0.2,
                                        'time_to_begin_plotting_null_arc': None,
                                        'time_to_end_plotting_null_arc': None,
                                        'max_num_plot_to_make': max_num_plot_to_make,
                                        'show_intended_target': False,
                                        'show_ff_indices': False,
                                        'point_indices_to_plot_null_arc': point_indices_to_plot_null_arc,
                                        'ff_indices_to_plot_null_arc': ff_indices_to_plot_null_arc,
                                        'whether_mark_path_where_intended_target_has_best_arc_among_all_ff': False}



show_null_trajectory.make_pretty_plot_for_a_duration(duration_to_plot, *pretty_plot_args, pretty_null_arc_plot_kwargs_temp, **additional_function_kwargs)

compare_eye_positions(monkey_information, duration_to_plot, pretty_plot_args, pretty_null_arc_plot_kwargs_temp, additional_function_kwargs, additional_plotting_kwargs)
          


# Other plots/info

## Distribution of change in angle over distance

In [ ]:
reload(curvature_utils)

data_item.make_curvature_df([-1, 1], clean=False)
data_item.make_best_arc_df()
best_arc_df = data_item.best_arc_df.copy()

In [ ]:
dist_of_diff = curvature_df['curv_diff'].copy().values

# use degrees 
dist_of_diff = dist_of_diff * 180 / math.pi

curvature_df['curv_diff'].describe()

In [ ]:
sns.histplot(dist_of_diff, bins=100)
plt.xlabel("Curvature difference")
plt.ylabel("Number of arcs")
plt.title("Histogram of Change in Angle Over 100 cm for each arc")
plt.show()


In [ ]:
sns.histplot(dist_of_diff, binwidth=0.5)
plt.xlim(-5, 5)
plt.xlabel("Curvature difference")
plt.ylabel("Number of arcs")
plt.title("Histogram of Change in Angle Over 100 cm for each arc")
plt.show()


In [ ]:
# See outliers
large_outliers = dist_of_diff[dist_of_diff > 5]
print('The percentage of large outliers is', round(len(large_outliers)/len(dist_of_diff)*100, 3), '%')

In [ ]:
small_outliers = dist_of_diff[dist_of_diff < -5]
print('The percentage of small outliers is', round(len(small_outliers)/len(dist_of_diff)*100, 3), '%')


## Now, remove outliers to calculate sd

In [ ]:
dist_w_outliers_removed = dist_of_diff[(dist_of_diff < 5) & (dist_of_diff > -5)]
new_var_of_diff = np.var(dist_w_outliers_removed)
new_var_of_diff

In [ ]:
np.sqrt(new_var_of_diff)

## Use percentage of abs_diff for error bar

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(best_arc_df.diff_percentile, bins=100)
plt.show()


## did abs_diff... change?

In [ ]:
best_arc_df.head(2)

In [ ]:
best_arc_df_sub = best_arc_df[best_arc_df['monkey_passed_by'] == True].copy()
best_arc_df_sub.sort_values(by=['intended_target_id', 'time'], inplace=True)

chunk_size = 10
unique_id = np.unique(best_arc_df_sub.intended_target_id.values)
num_chunks = math.ceil(len(unique_id)/chunk_size)
print('num_chunks', num_chunks)
plt.figure(figsize=(10, 7))

#for chunk in range(num_chunks):
for chunk in range(3,10):
    for i in range(len(unique_id))[chunk*chunk_size:(chunk+1)*chunk_size]:
        id = unique_id[i]
        best_arc_df_relevant = best_arc_df_sub[best_arc_df_sub.intended_target_id == id].copy()
        best_arc_df_relevant['time'] = best_arc_df_relevant['time'] - best_arc_df_relevant['time'].min()
        plt.plot(best_arc_df_relevant['time'], best_arc_df_relevant['curv_diff'], label=id)
        if i == 100:
            break
    plt.legend()
    plt.show()
    


## monkey_passed_by

In [ ]:
monkey_passed_by = best_arc_df['monkey_passed_by'].values
# what percentage of monkey_stopped_by is True?
np.sum(monkey_passed_by) / len(monkey_passed_by)

In [ ]:
unique_monkey_passed_by = best_arc_df[['intended_target_id', 'monkey_passed_by']].drop_duplicates()['monkey_passed_by'].values
# what percentage of unique_monkey_passed_by is True?
np.sum(unique_monkey_passed_by) / len(unique_monkey_passed_by)

## eliminate the case where monkey didn't pass by 

In [ ]:
best_arc_df_sub = best_arc_df[best_arc_df['monkey_passed_by'] == True].copy()

In [ ]:
not_walked_by = best_arc_df_sub[['point_index', 'ff_index']].copy()
not_walked_by['not_walked_by'] = True

In [ ]:
curvature_df_sub = curvature_df.merge(not_walked_by, how='left', on=['point_index', 'ff_index'])

In [ ]:
curvature_df_valid = curvature_df_sub[curvature_df_sub['not_walked_by'] != True]

In [ ]:
best_arc_df_sub

In [ ]:
best_arc_df_sub[best_arc_df_sub['intended_target_id'] == 1002].head(2)

In [ ]:
curvature_df

In [ ]:
curvature_df.copy().head(2)

In [ ]:
best_arc_df

In [ ]:
best_arc_df[best_arc_df['monkey_passed_by'] == True]

## curv auto correlation

In [ ]:
# find autocorrelation in curv_of_traj
monkey_information[monkey_information['time'] > ff_caught_T_new[-1]]

curv_of_traj_mode='time'
for half_window in np.arange(1, 10)*0.1:

    
    window_for_curv_of_traj=[-half_window, half_window]
    truncate_curv_of_traj_by_time_of_capture=False
    curv_of_traj_df, traj_curv_descr = curv_of_traj_utils.find_curv_of_traj_df_based_on_curv_of_traj_mode(window_for_curv_of_traj, monkey_information, ff_caught_T_new, curv_of_traj_mode=curv_of_traj_mode, truncate_curv_of_traj_by_time_of_capture=truncate_curv_of_traj_by_time_of_capture)
    curv_of_traj = curv_of_traj_df.curv_of_traj.values


    plot_acf(curv_of_traj, lags=50)
    plt.title("window_for_curv_of_traj = ", window_for_curv_of_traj)
    plt.show()
    

## monkey's reaction time

### visual (need to select valid cases)

In [ ]:
# find points where monkey is moving fast and turning fast
monkey_information_sub = monkey_information.loc[monkey_information['monkey_speed'] >= 20].copy()
monkey_information_sub['abs_monkey_ddw'] = np.abs(monkey_information_sub['monkey_ddw'])
monkey_information_sub = monkey_information_sub[monkey_information_sub['abs_monkey_ddw'] > 0.2].copy()

# also make sure that there's no stop right before each point within 1s (about 60 points) to the past and 0.5s to the future
# if there is, then take out that point
useful_index = []
for point_index in monkey_information_sub.point_index.values:
    info_sub = monkey_information.loc[point_index-60 : point_index+30].copy()
    if (info_sub['monkey_speeddummy']==0).sum() == 0:
        useful_index.append(point_index)
monkey_information_sub2 = monkey_information.loc[np.array(useful_index)]

In [ ]:
current_i = 10

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7] 
point_samples = monkey_information_sub2.index.values[::]
time_samples = monkey_information_sub2.time.values[::]
max_num_plot_to_make = 10

max_num_plot_to_make = 3
plot_counter = 0
for i in range(len(time_samples))[current_i:]:
    point_index = point_samples[i]    
    time = time_samples[i]
    print('point_index: ', point_index)  
    additional_plotting_kwargs = {'point_indices_to_be_marked_2nd_kind': point_index}

    fig, axes = show_null_trajectory.make_plots_to_show_monkey_reaction_time(curvature_df,                    
                                                                            null_arcs_plotting_kwargs,
                                                                            data_item.PlotTrials_args
                                                                            time_samples, 
                                                                            point_samples,
                                                                            show_percentile_by_color = False,
                                                                            additional_plotting_kwargs=additional_plotting_kwargs)

    axes = fig.axes[0]
    axes.set_title(point_index)
      
    plt.show()
    
    plot_counter += 1
    current_i = i
    if plot_counter == max_num_plot_to_make:
        break

### use valid cases

In [ ]:
# the points in index_array are found while inspecting the plots from before

### see stats of time difference between the high speed/turning points and the earliest time point when a ff has been visible (has a valid null arc) in the past 1s
index_array = np.array([39452, 57299, 60314, 61698, 64730, 66697, 70425, 79691, 83832, 88828, 114760, 129392])
diff_time_array = []
for index in index_array:
    time = monkey_information.loc[index, 'time'].item()
    curvature_df_sub = curvature_df[curvature_df.time.between(time-1, time)]
    min_time = curvature_df_sub.time.min()
    diff_time = time-min_time
    diff_time_array.append(diff_time)
diff_time_array = np.array(diff_time_array)
monkey_sub = monkey_information.loc[index_array].copy()
monkey_sub['diff_time'] = diff_time_array
monkey_sub.sort_values(by='diff_time', ascending=True, inplace=True)

#### visual (of valid cases)

In [ ]:
current_i = 0

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7] 
point_samples = monkey_information_sub2.index.values[::]
time_samples = monkey_information_sub2.time.values[::]
max_num_plot_to_make = 10

max_num_plot_to_make = 3
plot_counter = 0
for i in range(len(time_samples))[current_i:]:
    point_index = point_samples[i]    
    time = time_samples[i]
    print('point_index: ', point_index)  
    additional_plotting_kwargs = {'point_indices_to_be_marked_2nd_kind': point_index}

    fig, axes = show_null_trajectory.make_plots_to_show_monkey_reaction_time(curvature_df,                    
                                                                            null_arcs_plotting_kwargs,
                                                                            data_item.PlotTrials_args,
                                                                            time, 
                                                                            show_percentile_by_color = False,
                                                                            additional_plotting_kwargs=additional_plotting_kwargs)


    plt.show()
    
    plot_counter += 1
    current_i = i
    if plot_counter == max_num_plot_to_make:
        break

In [ ]:
point_samples = monkey_sub.index.values
time_samples = monkey_sub.time.values
diff_time_samples = monkey_sub.diff_time.values

max_num_plot_to_make = 3
plot_counter = 0
for i in range(len(time_samples))[current_i:]:
    point_index = point_samples[i]    
    time = time_samples[i]
    print('point_index: ', point_index)  
    additional_plotting_kwargs = {'point_indices_to_be_marked_2nd_kind': point_index}

    fig, axes = show_null_trajectory.make_plots_to_show_monkey_reaction_time(curvature_df,                    
                                                                            null_arcs_plotting_kwargs,
                                                                            data_item.PlotTrials_args,
                                                                            time, 
                                                                            show_percentile_by_color = False,
                                                                            additional_plotting_kwargs=additional_plotting_kwargs)
    
    axes = fig.axes[0]
    axes.set_title(point_index)
      
    plt.show()
    
    plot_counter += 1
    current_i = i
    if plot_counter == max_num_plot_to_make:
        break

### stats

In [ ]:
# distribution of diff_time among valid cases
sns.histplot(diff_time_array, binwidth=0.01)
plt.show()


In [ ]:
sns.histplot(monkey_information_sub['monkey_speed'], bins=40, stat='probability')
plt.show()


sns.boxplot(monkey_information_sub['monkey_speed'].values)
plt.show()


In [ ]:
# for every big ddw (but also big monkey_speed, cause otherwise the monkey might have just stopped)
sns.histplot(monkey_information_sub['monkey_ddw'], bins=40, stat='probability')
plt.show()


sns.boxplot(monkey_information_sub['monkey_ddw'].values.copy())
plt.show()

# see if there's an intended target that has just flashed on not long ago...

# Compared to manual_anno_long

## best_arc_df

In [ ]:
reload(curvature_utils)

reload(find_best_arc)
reload(intended_targets_classes)
reload(show_null_trajectory)


data_item.make_curvature_df([-25, 25])
data_item.find_opt_arc_info_from_curvature_df()
data_item.make_best_arc_df() # this is the df for plotting
data_item.get_elements_for_plotting()

best_arc_df = data_item.best_arc_df

In [ ]:
best_arc_original_columns = data_item.best_arc_original_columns

## get manual_anno

In [ ]:
reload(pattern_by_trials)
reload(decision_making_class)
reload(decision_making_utils)
reload(show_null_trajectory)
reload(intended_targets_classes)

keeping_1_out_of_n_rows = 1
num_time_points_for_trajectory = 100

time_range_of_trajectory = [-1, 1]
ff_dataframe_truncated = ff_dataframe[ff_dataframe['time_since_last_vis'] <= 2.5]

moit = intended_targets_classes.ModelOfIntendedTargets(raw_data_folder_path=raw_data_folder_path, time_range_of_trajectory=time_range_of_trajectory, num_time_points_for_trajectory=num_time_points_for_trajectory)
moit.retrieve_manual_anno() 
moit.get_and_process_manual_anno_long(n_seconds_after_crossing_boundary=2.5)
moit.get_input_data(num_ff_per_row=6, keeping_1_out_of_n_rows=keeping_1_out_of_n_rows)

chosen_rows_of_df = moit.chosen_rows_of_df
sequence_of_obs_ff_indices = moit.sequence_of_obs_ff_indices

moit.prepare_data_for_machine_learning(furnish_with_trajectory_data=False, trajectory_data_kind="position")
#moit.turn_y_label_into_multi_class(manual_anno_mul = None, allow_multi_label = False)
moit.split_data_to_train_and_test(scaling_data=True, keep_whole_chunks=False)
moit.use_machine_learning_model(model=None)

In [ ]:
free_selection_x_df = moit.free_selection_x_df
free_selection_labels = moit.free_selection_labels
free_selection_time = moit.free_selection_time
cases_for_inspection = moit.cases_for_inspection
sequence_of_obs_ff_indices = moit.sequence_of_obs_ff_indices
chosen_rows_of_df = moit.chosen_rows_of_df
non_chosen_rows_of_df = moit.non_chosen_rows_of_df

## combine info

In [ ]:
reload(find_best_arc)
chosen_rows_merged, best_arc_pred, mismatched_rows, mismatched_indices, no_match_rows = find_best_arc.combine_manual_anno_and_best_arc_df_info_for_comparison(best_arc_df, chosen_rows_of_df, sequence_of_obs_ff_indices)

In [ ]:
print('Percentage of correct match:', 1-len(mismatched_rows)/(len(chosen_rows_merged)))

In [ ]:
# let's see among the rows where best_arc_ff_index is not negative, how many of them are actually wrong
chosen_rows_merged_sub = chosen_rows_merged[chosen_rows_merged['best_arc_ff_index'] >= 0]
mismatched_rows_sub = chosen_rows_merged_sub[chosen_rows_merged_sub['best_arc_ff_index'] != chosen_rows_merged_sub['anno_ff_index']]

print('Among the rows where best_arc_ff_index is not negative, the percentage of correct match is:', 1-len(mismatched_rows_sub)/(len(chosen_rows_merged_sub)))

## plot arcs

In [ ]:
reload(curvature_utils)

reload(find_best_arc)

In [ ]:
moit.prepare_data_for_machine_learning(furnish_with_trajectory_data=True, trajectory_data_kind="position")
#moit.turn_y_label_into_multi_class(manual_anno_mul = None, allow_multi_label = False)
moit.split_data_to_train_and_test(scaling_data=True, keep_whole_chunks=False)
moit.use_machine_learning_model(model=GaussianNB())

In [ ]:
# Get info to plot null arcs

best_arc_df_temp = data_item.curvature_df.copy().rename(columns={'opt_arc_radius': 'arc_radius',  'opt_arc_measure': 'arc_measure'})
null_arc_info_for_plotting_in_polar = show_null_trajectory.find_and_package_opt_arc_info_for_plotting(best_arc_df_temp, mode='polar')


# Get info to plot triangles to show monkey directions
traj_time_2d, trajectory_data_dict = trajectory_info.find_trajectory_data(\
                moit.time_all, moit.monkey_information, time_range_of_trajectory = moit.time_range_of_trajectory, num_time_points_for_trajectory=moit.num_time_points_for_trajectory)
traj_x_2d = trajectory_data_dict['monkey_x']
traj_y_2d = trajectory_data_dict['monkey_y']
traj_dv_2d = trajectory_data_dict['monkey_speed']
traj_dw_2d = trajectory_data_dict['monkey_dw']
monkey_angle_2d = trajectory_data_dict['monkey_angle']
monkey_indices = np.searchsorted(moit.monkey_information['time'], moit.time_all) 
traj_distances, traj_angles, monkey_angle_on_trajectory_relative_to_the_current_north = trajectory_info.find_monkey_info_on_trajectory_relative_to_origin(monkey_indices, moit.monkey_information, traj_x_2d, traj_y_2d, monkey_angle_2d, num_time_points_for_trajectory=moit.num_time_points_for_trajectory)
traj_x, traj_y = animation_utils.change_polar_to_xy(traj_distances, traj_angles)
left_end_x, left_end_y, right_end_x, right_end_y = animation_utils.find_triangle_to_show_direction(traj_x, traj_y, monkey_angle_on_trajectory_relative_to_the_current_north)
# now, change the end points back to polar
left_end_r, left_end_theta = animation_utils.change_xy_to_polar(left_end_x, left_end_y)
right_end_r, right_end_theta = animation_utils.change_xy_to_polar(right_end_x, right_end_y)

mheading = {'traj_r': traj_distances[selected_cases], 'traj_theta': traj_angles[selected_cases],
                            'left_end_r': left_end_r[selected_cases], 'left_end_theta': left_end_theta[selected_cases],
                            'right_end_r': right_end_r[selected_cases], 'right_end_theta': right_end_theta[selected_cases]}



current_i = 0

In [ ]:
# In the function below, predicted ff means ff predicted by best_arc, and real ff means ff marked by manual anno

max_plot_to_make = 2
selected_cases = mismatched_indices[range(0, len(mismatched_indices), 10)][current_i:]
#selected_cases = no_match_rows
current_i += max_plot_to_make

moit.prepare_to_plot_prediction_results()
polar_plots_kwargs = moit.polar_plots_kwargs.copy()
polar_plots_kwargs['y_pred'] = best_arc_pred.copy()
polar_plots_kwargs['show_direction_of_monkey_on_trajectory'] = True

null_arcs_bundle = dict()
null_arcs_bundle['null_arc_info_for_plotting'] = null_arc_info_for_plotting_in_polar.copy()
null_arcs_bundle['all_point_index'] = moit.free_selection_point_index.copy()

plot_decision_making.make_polar_plots_for_decision_making(**moit.polar_plots_kwargs,
                                                          null_arcs_bundle=null_arcs_bundle)

In [ ]:
stop!

## animation

In [ ]:
reload(decision_making_utils)
moit.y_pred = best_arc_pred
anno_ff_indices_dict = decision_making_utils.make_anno_ff_indices_dict(moit)


In [ ]:
# also make pred_ff_indices_dict
pred_ff_indices_dict = {}
pred_ff_colors_dict = {}
viridis = matplotlib.colormaps['PiYG']
for index, row in best_arc_df.iterrows():
    if row['ff_index'] >= 0:
        pred_ff_indices_dict[int(row.point_index)] = int(row.ff_index)
        pred_ff_colors_dict[int(row.point_index)] = viridis(row.diff_percentile_in_decimal)

In [ ]:
# # also make pred_ff_indices_dict
# pred_ff_indices_dict = {}
# pred_ff_colors_dict = {}
# viridis = matplotlib.colormaps['viridis']
# for index, row in best_arc_df.iterrows():
#     if row['ff_index'] >= 0:
#         pred_ff_indices_dict[int(row.point_index)] = int(row.ff_index)
#         pred_ff_colors_dict[int(row.point_index)] = viridis(row.diff_percentile_in_decimal)

In [ ]:
temp_animation_plot_kwargs = data_item.animation_plot_kwargs.copy()
temp_animation_plot_kwargs['images_dir'] = None
temp_animation_plot_kwargs['show_stops'] = False
temp_animation_plot_kwargs['trail_color_var'] = "grey"
plt.rcParams["figure.figsize"] = (10, 10)

In [ ]:
# the bluer the better (more confidence), I think 

In [ ]:
plt.rcParams['animation.ffmpeg_path'] = './ffmpeg'
plt.rcParams["figure.figsize"] = (12, 12)


#for i in range(math.ceil(moit.free_selection_time.min()), math.floor(moit.free_selection_time.max()), 25)[current_i:]:
for i in range(math.ceil(moit.free_selection_time.min())+1, math.floor(moit.free_selection_time.max()), 25)[current_i:current_i+10]:
    current_i += 1
    #duration = [i, i+5]
    duration = [i, i+30]
    print(duration)
    video_dir = "/Users/dusiyi/Documents/Multifirefly-Project/for_manual_annotation_7"
    #os.makedirs(video_dir, exist_ok = True)
    file_name = f"time_{duration[0]}_to_{duration[1]}.mp4"
    k = 3
    data_item.make_animation(duration=duration, save_video=True, video_dir=video_dir, file_name=file_name, show_ff_indices=True, 
                                static_plot_on_the_left=False, animation_plot_kwargs=temp_animation_plot_kwargs, plot_time_index=True, show_speed_through_path_color=True,
                                max_num_frames=None, max_duration=None, min_duration=1, set_xy_limits=True, anno_ff_indices_dict=anno_ff_indices_dict, 
                                pred_ff_indices_dict=pred_ff_indices_dict, pred_ff_colors_dict=pred_ff_colors_dict,
                                k=k, fps=int(62/k/2), margin=150, monkey_information=monkey_information,
                                rotated=True, show_direction_through_triangle=True, show_full_arena=True) # slow down the video by 2x
                                # let's increase the margin to 150 and see how it goes
    HTML(data_item.anim.to_html5_video()) 


## valid ff_index only
Also, we want not just chosen rows, but all data that have valid ff_index

In [ ]:
best_arc_df_sub = best_arc_df[['point_index', 'ff_index']].copy()
best_arc_df_sub = best_arc_df_sub[best_arc_df_sub['ff_index'] >= 0]
best_arc_df_sub.rename(columns={'ff_index': 'best_arc_ff_index'}, inplace=True)

# get manual_anno_long
manual_anno_long_sub = moit.manual_anno_long[['starting_point_index', 'ff_index']].copy()
manual_anno_long_sub = manual_anno_long_sub[manual_anno_long_sub['ff_index'] >= 0]
manual_anno_long_sub.rename(columns={'starting_point_index': 'point_index', 'ff_index': 'anno_ff_index'}, inplace=True)


merged_valid_ff_df = pd.merge(best_arc_df_sub, manual_anno_long_sub, how='inner', on=['point_index'])

merged_valid_ff_df['matched'] = (merged_valid_ff_df['anno_ff_index'] == merged_valid_ff_df['best_arc_ff_index']).astype('int')


# calculate the percentage of match, mismatch, and no_match
print('Percentage of matched pairs among point_indices where both methods produce valid ff_index:', merged_valid_ff_df['matched'].sum()/len(merged_valid_ff_df))


In [ ]:
mismatch_point_indices = merged_valid_ff_df[merged_valid_ff_df['matched']==0].point_index.values
# plot a histogram for mismatch_point_indices
plt.figure(figsize=(10, 7))
plt.hist(mismatch_point_indices, bins=100)
plt.show()



## inspect some chunks of time

In [ ]:
# plot the chunks
reload(plot_trials)
plt.figure(figsize=(12, 12))

plot_chunks_kwargs_temp = plot_chunks_kwargs.copy()
plot_chunks_kwargs_temp['show_believed_target_positions'] = False
plot_chunks_kwargs_temp['show_connect_path_ff'] = False
plot_chunks_kwargs_temp['show_alive_fireflies'] = False
plot_chunks_kwargs_temp['zoom_in'] = True
plot_chunks_kwargs_temp['show_trajectory'] = True
plot_chunks_kwargs_temp['trail_color_var'] = 'time'
for time in [1430.950184]:
  with general_utils.initiate_plot(7, 7, 100):
    duration = [time+30*0.016, time+60*0.016]
    print("duration", duration)
    returned_info = plot_trials.PlotTrials(duration,
               *PlotTrials_args,
               **plot_chunks_kwargs_temp,
               show_monkey_angles=True)
    plt.show()
    

# monkey angle calculation experiments

## different add_monkey_angles_column functions

### go with smaller one

In [ ]:
def calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past):
    delta_x = monkey_information['monkey_x'].iloc[i+num_points_future-1] - monkey_information['monkey_x'].iloc[i-num_points_past-1]
    delta_y = monkey_information['monkey_y'].iloc[i+num_points_future-1] - monkey_information['monkey_y'].iloc[i-num_points_past-1]
    current_delta_position = np.sqrt(np.square(delta_x)+np.square(delta_y))
    return delta_x, delta_y, current_delta_position


def calculate_monkey_angles_small(monkey_information, min_distance_to_calculate_angle=5):
    # Add angle of the monkey
    monkey_angles = [pi/2]  # The monkey is at 90 degree angle at the beginning
    current_angle = pi/2 # This keeps track of the current angle during the iterations

    delta_x = np.diff(monkey_information['monkey_x'])
    delta_y = np.diff(monkey_information['monkey_y'])
    delta_position = np.sqrt(np.square(delta_x) + np.square(delta_y))

    # Find the time in the data that is closest (right before) the time where we wan to know the monkey's angular position.
    total_points = len(monkey_information['time'])
    num_points_past = 0
    num_points_future = 1
    delta_position_full_length = np.append(0, delta_position)
    for i in range(1, total_points):
        if i+(num_points_past+num_points_future) >= total_points:
            monkey_angles.extend([current_angle] * (total_points-i))
            break
        # if i%10000 == 0:
        #     print(i, "out of,", total_points, "for calculating monkey_angles")
        current_delta_position = delta_position_full_length[i]
        if current_delta_position > min_distance_to_calculate_angle:
            num_points_past = 0
            num_points_future = 1
        else:
            num_points_past = max(0, num_points_past-1)
            num_points_future = max(1, num_points_future-1)
            
            delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
            while (current_delta_position <= min_distance_to_calculate_angle) and (i+num_points_past+num_points_future < total_points):
                # try to see if adding 1 to num_points_future produce smaller increase in delta_position or adding 1 to num_points_past
                # the reason we want to go with the smaller increase is because we want to jump out of a monkey stop where there is a lot of noise
                delta_x, delta_y, current_delta_position_future = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future+1, num_points_past)
                delta_x, delta_y, current_delta_position_past = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past+1)
                if current_delta_position_future <= current_delta_position_past:
                    num_points_future += 1
                else:
                    num_points_past += 1
                delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
        if current_delta_position < 50:
            delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
            # calculate the angle defined by two points
            current_angle = math.atan2(delta_y, delta_x)
        else:
            current_angle = current_angle
        print('time', monkey_information['time'].iloc[i], 'num_points', num_points_past+num_points_future, 'angle', current_angle, 'current_delta_position', current_delta_position, 'dy', delta_y, 'dx', delta_x)
        # Otherwise, most likely the monkey has crossed the boundary and come out at another place; we shall keep the current angle, and not update it
        monkey_angles.append(current_angle)
    monkey_information['monkey_angle'] = np.array(monkey_angles)
    return monkey_information

test_calculate_monkey_angles_function(time, monkey_information)

### go with bigger one

In [ ]:
def calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past):
    delta_x = monkey_information['monkey_x'].iloc[i+num_points_future-1] - monkey_information['monkey_x'].iloc[i-num_points_past-1]
    delta_y = monkey_information['monkey_y'].iloc[i+num_points_future-1] - monkey_information['monkey_y'].iloc[i-num_points_past-1]
    current_delta_position = np.sqrt(np.square(delta_x)+np.square(delta_y))
    return delta_x, delta_y, current_delta_position


def calculate_monkey_angles_big(monkey_information, min_distance_to_calculate_angle=5):
    # Add angle of the monkey
    monkey_angles = [pi/2]  # The monkey is at 90 degree angle at the beginning
    current_angle = pi/2 # This keeps track of the current angle during the iterations

    delta_x = np.diff(monkey_information['monkey_x'])
    delta_y = np.diff(monkey_information['monkey_y'])
    delta_position = np.sqrt(np.square(delta_x) + np.square(delta_y))

    # Find the time in the data that is closest (right before) the time where we wan to know the monkey's angular position.
    total_points = len(monkey_information['time'])
    num_points_past = 0
    num_points_future = 1
    delta_position_full_length = np.append(0, delta_position)
    for i in range(1, total_points):
        if i+(num_points_past+num_points_future) >= total_points:
            monkey_angles.extend([current_angle] * (total_points-i))
            break
        # if i%10000 == 0:
        #     print(i, "out of,", total_points, "for calculating monkey_angles")
        current_delta_position = delta_position_full_length[i]
        if current_delta_position > min_distance_to_calculate_angle:
            num_points_past = 0
            num_points_future = 1
        else:
            num_points_past = max(0, num_points_past-1)
            num_points_future = max(1, num_points_future-1)
            
            delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
            while (current_delta_position <= min_distance_to_calculate_angle) and (i+num_points_past+num_points_future < total_points):
                # try to see if adding 1 to num_points_future produce smaller increase in delta_position or adding 1 to num_points_past
                # the reason we want to go with the smaller increase is because we want to jump out of a monkey stop where there is a lot of noise
                delta_x, delta_y, current_delta_position_future = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future+1, num_points_past)
                delta_x, delta_y, current_delta_position_past = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past+1)
                if current_delta_position_future >= current_delta_position_past:
                    num_points_future += 1
                else:
                    num_points_past += 1
                delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
        if current_delta_position < 50:
            delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
            # calculate the angle defined by two points
            current_angle = math.atan2(delta_y, delta_x)
        else:
            current_angle = current_angle
        print('time', monkey_information['time'].iloc[i], 'num_points', num_points_past+num_points_future, 'angle', current_angle, 'current_delta_position', current_delta_position, 'dy', delta_y, 'dx', delta_x)
        # Otherwise, most likely the monkey has crossed the boundary and come out at another place; we shall keep the current angle, and not update it
        monkey_angles.append(current_angle)
    monkey_information['monkey_angle'] = np.array(monkey_angles)
    return monkey_information

test_calculate_monkey_angles_function(time, monkey_information)

### equal (currently used)

In [ ]:
def calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past):
    delta_x = monkey_information['monkey_x'].iloc[i+num_points_future-1] - monkey_information['monkey_x'].iloc[i-num_points_past-1]
    delta_y = monkey_information['monkey_y'].iloc[i+num_points_future-1] - monkey_information['monkey_y'].iloc[i-num_points_past-1]
    current_delta_position = np.sqrt(np.square(delta_x)+np.square(delta_y))
    return delta_x, delta_y, current_delta_position


def calculate_monkey_angles_equal(monkey_information, min_distance_to_calculate_angle=5):
    # Add angle of the monkey
    monkey_angles = [pi/2]  # The monkey is at 90 degree angle at the beginning
    current_angle = pi/2 # This keeps track of the current angle during the iterations

    delta_x = np.diff(monkey_information['monkey_x'])
    delta_y = np.diff(monkey_information['monkey_y'])
    delta_position = np.sqrt(np.square(delta_x) + np.square(delta_y))

    # Find the time in the data that is closest (right before) the time where we wan to know the monkey's angular position.
    total_points = len(monkey_information['time'])
    num_points = 1
    delta_position_full_length = np.append(0, delta_position)
    for i in range(1, total_points):
        if i+num_points >= total_points:
            monkey_angles.extend([current_angle] * (total_points-i))
            break
        # if i%10000 == 0:
        #     print(i, "out of,", total_points, "for calculating monkey_angles")
        current_delta_position = delta_position_full_length[i]
        if current_delta_position > min_distance_to_calculate_angle:
            num_points = 1
        else:
            if num_points >= 3:
                num_points -= 2 # compared to i-1, i might need at least num_points-1 points into the future to accumulate enough distance; since there's num_points+=1 in while loop, here we shall minus 2
            else: 
                num_points = 1
            while (current_delta_position <= min_distance_to_calculate_angle) and (i+num_points < total_points):
                # num_points should be 2 or above to be meaningful
                num_points += 1 
                # now distribute the num_points to the past and future
                num_points_past = int(np.floor(num_points/2))
                num_points_future = num_points - num_points_past
                delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
        if current_delta_position < 50:
            # calculate the angle defined by two points
            delta_x, delta_y, current_delta_position = calculate_delta_xy_and_current_delta_position(monkey_information, i, num_points_future, num_points_past)
            # calculate the angle defined by two points
            current_angle = math.atan2(delta_y, delta_x)
        # Otherwise, most likely the monkey has crossed the boundary and come out at another place; we shall keep the current angle, and not update it
        monkey_angles.append(current_angle)
    monkey_information['monkey_angle'] = np.array(monkey_angles)
    return monkey_information

test_calculate_monkey_angles_function(time, monkey_information)

### original

In [ ]:
def calculate_monkey_angles_original(monkey_information, min_distance_to_calculate_angle=5):
    # Add angle of the monkey
    monkey_angles = [pi/2]  # The monkey is at 90 degree angle at the beginning
    current_angle = pi/2 # This keeps track of the current angle during the iterations

    delta_x = np.diff(monkey_information['monkey_x'])
    delta_y = np.diff(monkey_information['monkey_y'])
    delta_position = np.sqrt(np.square(delta_x) + np.square(delta_y))

    # Find the time in the data that is closest (right before) the time where we wan to know the monkey's angular position.
    total_points = len(monkey_information['time'])
    num_points = 1
    delta_position_full_length = np.append(0, delta_position)
    for i in range(1, total_points):
        if i+num_points >= total_points:
            monkey_angles.extend([current_angle] * (total_points-i))
            break
        # if i%10000 == 0:
        #     print(i, "out of,", total_points, "for calculating monkey_angles")
        current_delta_position = delta_position_full_length[i]
        if current_delta_position > min_distance_to_calculate_angle:
            num_points = 1
        else:
            if num_points >= 3:
                num_points -= 2 # compared to i-1, i might need at least num_points-1 points into the future to accumulate enough distance; since there's num_points+=1 in while loop, here we shall minus 2
            else:
                num_points = 1
            while (current_delta_position <= min_distance_to_calculate_angle) and (i+num_points < total_points):
                # num_points should be 2 or above to be meaningful
                num_points += 1 
                delta_x = monkey_information['monkey_x'].iloc[i+num_points-1] - monkey_information['monkey_x'].iloc[i-1]
                delta_y = monkey_information['monkey_y'].iloc[i+num_points-1] - monkey_information['monkey_y'].iloc[i-1]
                current_delta_position = np.sqrt(np.square(delta_x)+np.square(delta_y))
        if current_delta_position < 50:
            # calculate the angle defined by two points
            current_angle = math.atan2(monkey_information['monkey_y'].iloc[i+num_points-1]-monkey_information['monkey_y'].iloc[i-1], 
                                    monkey_information['monkey_x'].iloc[i+num_points-1]-monkey_information['monkey_x'].iloc[i-1])
        # Otherwise, most likely the monkey has crossed the boundary and come out at another place; we shall keep the current angle, and not update it
        monkey_angles.append(current_angle)
    monkey_information['monkey_angle'] = np.array(monkey_angles)
    return monkey_information



test_calculate_monkey_angles_function(time, monkey_information)

## compare different functions

In [ ]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        

def simultaneous_test_calculate_monkey_angles_function(time, monkey_information, min_distance_to_calculate_angle=5):
    with HiddenPrints():
        half_duration = 3
        monkey_information_new = monkey_information[monkey_information['time'].between(time-half_duration, time+half_duration)].copy()
        monkey_information_big = calculate_monkey_angles_big(monkey_information_new, min_distance_to_calculate_angle=min_distance_to_calculate_angle)

        monkey_information_new = monkey_information[monkey_information['time'].between(time-half_duration, time+half_duration)].copy()
        monkey_information_small = calculate_monkey_angles_small(monkey_information_new, min_distance_to_calculate_angle=min_distance_to_calculate_angle)

        monkey_information_new = monkey_information[monkey_information['time'].between(time-half_duration, time+half_duration)].copy()
        monkey_information_equal = calculate_monkey_angles_equal(monkey_information_new, min_distance_to_calculate_angle=min_distance_to_calculate_angle)

        monkey_information_new = monkey_information[monkey_information['time'].between(time-half_duration, time+half_duration)].copy()
        monkey_information_original = calculate_monkey_angles_original(monkey_information_new, min_distance_to_calculate_angle=min_distance_to_calculate_angle)
    plt.figure(figsize=(10, 7))
    margin = 10
    plt.scatter(monkey_information_big.time[margin:-margin], monkey_information_big.monkey_angles[margin:-margin]*180/math.pi, s=20, marker='o', alpha=0.5, c='red')
    plt.scatter(monkey_information_small.time[margin:-margin], monkey_information_small.monkey_angles[margin:-margin]*180/math.pi, s=20, marker='*', alpha=0.5, c='orange')
    plt.scatter(monkey_information_equal.time[margin:-margin], monkey_information_equal.monkey_angles[margin:-margin]*180/math.pi, s=20, marker='^', alpha=0.5, c='blue')
    plt.scatter(monkey_information_original.time[margin:-margin], monkey_information_original.monkey_angles[margin:-margin]*180/math.pi, s=20, marker='s', alpha=0.5, c='green')
    plt.title("Angles")
    plt.show()
    


    plt.figure(figsize=(10, 7))
    plt.scatter(monkey_information_new.monkey_x[margin:-margin], monkey_information_new.monkey_y[margin:-margin], c = range(len(monkey_information_new.monkey_x[10:-10])), cmap = 'Reds', s=1)
    # let x and y axis have the same scale
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title("monkey_x")
    plt.show() 
    


    # plt.figure(figsize=(10, 7))
    # plt.scatter(monkey_information_new.time[10:-10], monkey_information_new.monkey_x[10:-10], s=1)
    # plt.title("monkey_x")
    # plt.show() 
    # 

    # plt.figure(figsize=(10, 7))
    # plt.scatter(monkey_information_new.time[10:-10], monkey_information_new.monkey_y[10:-10], s=1)
    # plt.title("monkey_y")
    # plt.show()
    # 

In [ ]:
current_i = 0

In [ ]:
stop_time = monkey_information[monkey_information['monkey_speeddummy']==0].time.values
for time in stop_time[range(1000, len(monkey_information), 100)[current_i: current_i + 20]]:
    simultaneous_test_calculate_monkey_angles_function(time, monkey_information, min_distance_to_calculate_angle=5)
current_i += 20

# ## curvature upper lower bounds

In [ ]:

# see clean_curvature_info function
max(curvature_df.curv_diff.values)
for column in ['curv_of_traj', 'curvature_lower_bound', 'curvature_upper_bound', 'opt_arc_curv']:
    sns.boxplot(curvature_df[column].values, orient='h')
    plt.title(column)
    plt.show()
    
   
    sns.histplot(curvature_df[column].values, bins=50)
    plt.title(column)
    plt.show()
    